# This notebook reproduces all figures and experimental results of the paper.

- the dataset ['20-newsgroups'](https://scikit-learn.org/0.19/datasets/twenty_newsgroups.html) is available from the scikit-learn package and can be downloaded via their dataset loader.
- the dataset 'classic3' is not readily available through any major code package, but has to be retrieved from any of a few sources on the internet and put into the /data folder. We used a copy of the dataset found [here](http://ir.dcs.gla.ac.uk/resources/test_collections/) (split into its three categories CISI, CRANFIELD and MEDLINE). 

# numerical evaluation of the Negentropy computation and approximation

- produces figures 1 and 2 of the manuscript.
- only calls core functions of the code package, no data involved.

In [ ]:
from vMFne.logpartition import gradΦ, invgradΦ, vMF_entropy_Φ, logχ, log_besseli, banerjee_44
from vMFne.negentropy import Ψ, Ψ_base, gradΨ, dΨ_base
from scipy.special import loggamma
import mpmath

import numpy as np
import matplotlib.pyplot as plt

Ds = [2, 10, 100, 1000][::-1] # reverse order for plotting reasons only

# Numerical ODE solver
Ψ0 = [0., 1e-6] # initial values for Ψ(0), Ψ'(0). Ψ(0) not that important, can be changed post-hoc.
t0 = 0.0        # starting point ||μ||=t0 for numerical integration. Should stay zero here !

# Newton-Raphson parameters
max_iter = 10
atol = 1e-12

# plotting
fontsize = 14
ticksize = 12
clrs = ['brown', 'b', 'orange', 'g'][::-1]
fig = plt.figure(figsize=(12, 6))
K = 100 # number of evaluation points ||μ[k]||, k=1,..,K between (exluding) 0 and (excluding) 1.
        # Note that these are forced checkpoints for the solver, the solver may use way more steps !

for i,D in enumerate(Ds):

    V = np.ones((K,D))/np.sqrt(D) # unit length vectors, some functions need vector-valued inputs

    μs_norm = np.linspace(0, 1, K+2)[1:-1]
    _, gradΨμ = Ψ(μs_norm.reshape(-1,1)*V, D, Ψ0=Ψ0, t0=t0, return_grad=True)

    κs = np.linalg.norm(gradΨμ,axis=-1)

    # re-compute ||μ|| = Φ'(||η||) to make sure H[η] is not at a disadvantage because η are bad
    # - if anything, μ are bad ! 
    μs_norm = np.linalg.norm(gradΦ(κs.reshape(-1,1)*V),axis=-1)
    Ψμ, gradΨμ = Ψ(μs_norm.reshape(-1,1)*V, D, Ψ0=Ψ0, t0=t0, return_grad=True, solve_delta=False)
    κs_est_0, diffs = invgradΦ(μs_norm,D,max_iter=0,atol=atol)
    κs_est, diffs = invgradΦ(μs_norm,D,max_iter=max_iter,atol=atol)

    # NegEntropy
    plt.subplot(2,3,1)
    H = vMF_entropy_Φ(κs_est.reshape(-1,1) * V)
    Ψμ0 = (D/2. - 1.) * np.log(2) + loggamma(D/2)
    negH = - H - logχ(D) - Ψμ0
    plt.semilogy(μs_norm, negH, '-', label='D='+str(D), color=clrs[i])
    plt.semilogy(μs_norm, Ψμ, '--', color=clrs[i])
    plt.ylabel(r'$\Psi(\mu)$', fontsize=fontsize)
    plt.yticks(fontsize=ticksize)
    plt.xticks(fontsize=ticksize)
    plt.legend(loc=4, fontsize=ticksize, frameon=False)

    # Gradient of NegEntropy
    plt.subplot(2,3,4)
    plt.semilogy(μs_norm, np.linalg.norm(gradΨμ,axis=-1), label='D='+str(D), color=clrs[i])
    plt.semilogy(μs_norm, κs_est, '--', color=clrs[i])
    plt.ylabel(r'$||\ \nabla\Psi(\mu)\ ||$', fontsize=fontsize)
    plt.yticks(fontsize=ticksize)
    plt.xticks(fontsize=ticksize)
    plt.xlabel(r'$||\mu||$', fontsize=fontsize)

    # error on Negentropy
    plt.subplot(2,3,2)    
    plt.semilogy(μs_norm, np.abs(negH - Ψμ), label='D='+str(D), color=clrs[i])
    plt.ylabel('abs. error', fontsize=fontsize)
    plt.yticks(fontsize=ticksize)
    plt.xticks(fontsize=ticksize)

    # error on gradients
    plt.subplot(2,3,5)    
    plt.semilogy(μs_norm, np.abs(np.linalg.norm(gradΨμ,axis=-1) - κs_est), '-',
                 label='D='+str(D), color=clrs[i])
    plt.ylabel('abs. error', fontsize=fontsize)
    plt.yticks(fontsize=ticksize)
    plt.xticks(fontsize=ticksize)
    plt.xlabel(r'$||\mu||$', fontsize=fontsize)

    # error on Negentropy
    plt.subplot(2,3,3)
    plt.semilogy(μs_norm, np.abs(negH - Ψμ)/negH, label='D='+str(D), color=clrs[i])
    plt.ylabel('rel. error', fontsize=fontsize)
    plt.yticks(fontsize=ticksize)
    plt.xticks(fontsize=ticksize)

    # error on gradients
    plt.subplot(2,3,6)    
    plt.semilogy(μs_norm, np.abs(np.linalg.norm(gradΨμ,axis=-1) - κs_est)/κs_est, '-',
                 label='D='+str(D), color=clrs[i])
    plt.xlabel(r'$||\mu||$', fontsize=fontsize)
    plt.ylabel('rel. error', fontsize=fontsize)
    plt.yticks(fontsize=ticksize)
    plt.xticks(fontsize=ticksize)

for i in range(6):
    ax = plt.subplot(2,3,i+1)
    ax.spines[['right', 'top']].set_visible(False)

plt.subplots_adjust(wspace=0.3)
plt.savefig('fig1.pdf', bbox_inches='tight')
plt.show()


In [ ]:
from vMFne.logpartition import gradΦ, invgradΦ, vMF_entropy_Φ, logχ, log_besseli, banerjee_44
from vMFne.negentropy import Ψ, Ψ_base, gradΨ, dΨ_base
from scipy.special import loggamma
import mpmath

import numpy as np
import matplotlib.pyplot as plt

Ds = [10, 100, 1000, 10000][::-1] # reverse order for plotting reasons only

# Numerical ODE solver
t0 = 0.0
Ψ0 = [0., 1e-5] # we're going to higher D's in this plot, so we increase Ψ'(0) to avoid underflows.

# plotting
clrs = ['b', 'orange', 'g', 'r'][::-1]
fontsize=14
ticksize=12
fig = plt.figure(figsize=(12, 9))
K = 100 # number of evaluation points ||μ[k]||, k=1,..,K between (exluding) 0 and (excluding) 1.

for i,D in enumerate(Ds):

    V = np.ones((K,D))/np.sqrt(D) # unit length vectors, some functions need vector-valued inputs
    μs_norm = np.linspace(0, 1, K+2)[1:-1]
    Ψμ, gradΨμ = Ψ(μs_norm.reshape(-1,1)*V, D, Ψ0=Ψ0, t0=t0, return_grad=True, solve_delta=True)

    # NegEntropy
    plt.subplot(3,3,1)
    plt.semilogy(μs_norm, Ψμ, label='D='+str(D), color=clrs[i])
    plt.ylabel(r'$\Psi(\mu)$', fontsize=fontsize)
    plt.yticks(fontsize=ticksize)
    plt.xticks(fontsize=ticksize)

    μ2 = μs_norm**2
    Ψμ44 = 0.5 *((1-D) * np.log(1 - μ2) + μ2)    

    # absolute error
    plt.subplot(3,3,2)
    plt.semilogy(μs_norm, np.abs(Ψμ - Ψ_base(μs_norm,D=D)), label='D='+str(D), color=clrs[i])
    plt.semilogy(μs_norm, np.abs(Ψμ - Ψμ44), '--', color=clrs[i])
    plt.ylabel(r'abs. error', fontsize=fontsize) # $(\Psi(\mu)- \tilde{\Psi}(\mu))/\Psi(\mu)$')
    plt.yticks(fontsize=ticksize)
    plt.xticks(fontsize=ticksize)
    plt.legend(loc=4, fontsize=ticksize, frameon=False)

    # relative error
    plt.subplot(3,3,3)
    plt.semilogy(μs_norm, np.abs(Ψμ - Ψ_base(μs_norm,D=D)) / Ψμ, label='D='+str(D), color=clrs[i])
    plt.semilogy(μs_norm, np.abs(Ψμ - Ψμ44) / Ψμ, '--', color=clrs[i])
    plt.ylabel(r'rel. error', fontsize=fontsize) # $(\Psi(\mu)- \tilde{\Psi}(\mu))/\Psi(\mu)$')
    plt.yticks(fontsize=ticksize)
    plt.xticks(fontsize=ticksize)

    # Gradient of NegEntropy
    plt.subplot(3,3,4)
    plt.semilogy(μs_norm, np.linalg.norm(gradΨμ,axis=-1), label='D='+str(D), color=clrs[i])
    plt.ylabel(r'$||\ \nabla\Psi(\mu)\ ||$', fontsize=fontsize)
    plt.yticks(fontsize=ticksize)
    plt.xticks(fontsize=ticksize)

    # absolute error
    plt.subplot(3,3,5)
    plt.semilogy(μs_norm, np.abs(np.linalg.norm(gradΨμ,axis=-1) - dΨ_base(μs_norm,D=D)),
                 label='D='+str(D), color=clrs[i])
    plt.semilogy(μs_norm, np.abs(np.linalg.norm(gradΨμ,axis=-1) - banerjee_44(μs_norm,D=D)),
                '--', color=clrs[i])
    plt.ylabel(r'abs. error', fontsize=fontsize) 
    plt.yticks(fontsize=ticksize)
    plt.xticks(fontsize=ticksize)

    # relative error
    plt.subplot(3,3,6)
    plt.semilogy(μs_norm, 
                 np.abs(np.linalg.norm(gradΨμ,axis=-1)-dΨ_base(μs_norm,D=D))/np.linalg.norm(gradΨμ,axis=-1),
                 label='D='+str(D), color=clrs[i])
    plt.semilogy(μs_norm, 
                 np.abs(np.linalg.norm(gradΨμ,axis=-1)-banerjee_44(μs_norm,D=D))/np.linalg.norm(gradΨμ,axis=-1),
                '--', color=clrs[i])
    plt.ylabel(r'rel. error', fontsize=fontsize)
    plt.yticks(fontsize=ticksize)
    plt.xticks(fontsize=ticksize)

    # Bregman div + const
    plt.subplot(3,3,7)
    dΨdμ_x_μ = μs_norm * np.linalg.norm(gradΨμ,axis=1) - Ψμ
    plt.semilogy(μs_norm, dΨdμ_x_μ, label='D='+str(D), color=clrs[i])
    plt.ylabel(r'$D_\Psi(0\ ||\ \mu)$', fontsize=fontsize)
    plt.yticks(fontsize=ticksize)
    plt.xticks(fontsize=ticksize)
    plt.xlabel(r'$||\mu||$', fontsize=fontsize)

    dΨdμ_x_μ_base = μs_norm * dΨ_base(μs_norm,D=D) - Ψ_base(μs_norm,D=D)
    μ2 = μs_norm**2
    Ψμ44 = 0.5 *((1-D) * np.log(1 - μ2) + μ2)
    dΨdμ_x_μ_44 = μs_norm * banerjee_44(μs_norm,D=D) - Ψμ44

    # absolute error
    plt.subplot(3,3,8)
    plt.semilogy(μs_norm, np.abs(dΨdμ_x_μ - dΨdμ_x_μ_base), label='D='+str(D), color=clrs[i])
    plt.semilogy(μs_norm, np.abs(dΨdμ_x_μ - dΨdμ_x_μ_44) , '--', color=clrs[i])
    plt.ylabel(r'abs. error', fontsize=fontsize)
    plt.xlabel(r'$||\mu||$', fontsize=fontsize)
    plt.yticks(fontsize=ticksize)
    plt.xticks(fontsize=ticksize)

    # relative error
    plt.subplot(3,3,9)
    plt.semilogy(μs_norm, np.abs(dΨdμ_x_μ - dΨdμ_x_μ_base) / dΨdμ_x_μ, label='D='+str(D), color=clrs[i])
    plt.semilogy(μs_norm, np.abs(dΨdμ_x_μ - dΨdμ_x_μ_44) / dΨdμ_x_μ, '--', color=clrs[i])
    plt.ylabel(r'rel. error', fontsize=fontsize)
    plt.xlabel(r'$||\mu||$', fontsize=fontsize)
    plt.yticks(fontsize=ticksize)
    plt.xticks(fontsize=ticksize)


"""
# closed-form approximations to those values - leaving out since one can't see any difference anyways !
for i,D in enumerate(Ds):
    dΨdμ_x_μ_base = μs_norm * dΨ_base(μs_norm,D=D) - Ψ_base(μs_norm,D=D)
    μ2 = μs_norm**2
    Ψμ44 = 0.5 *((1-D) * np.log(1 - μ2) + μ2)
    dΨdμ_x_μ_44 = μs_norm * banerjee_44(μs_norm,D=D) - Ψμ44

    plt.subplot(2,3,1)
    plt.plot(μs_norm, Ψ_base(μs_norm,D=D), ':', color='k')
    μ2 = μs_norm**2
    Ψμ44 = 0.5 *((1-D) * np.log(1 - μ2) + μ2)    
    plt.plot(μs_norm, Ψμ44, '--', color='gray')
    plt.subplot(2,3,2)
    plt.plot(μs_norm, dΨ_base(μs_norm,D=D), ':', color='k')
    plt.plot(μs_norm, banerjee_44(μs_norm,D=D), '--', color='gray')
    plt.subplot(2,3,3)
    plt.plot(μs_norm, dΨdμ_x_μ_base, ':', color='k')
    plt.plot(μs_norm, dΨdμ_x_μ_44, '--', color='gray')
"""

for i in range(9):
    ax = plt.subplot(3,3,i+1)
    ax.spines[['right', 'top']].set_visible(False)

plt.subplots_adjust(wspace=0.3) # more space horizontally between panels
plt.savefig('fig2.pdf', bbox_inches='tight')
plt.show()


In [ ]:
from vMFne.negentropy import Ψ, Ψ_base, gradΨ, dΨ_base

import numpy as np
import matplotlib.pyplot as plt
import jax.numpy as jnp
from jax import grad

D = 2000 #28571

# Numerical ODE solver
t0 = 0.0
Ψ0 = [0., 1e-4] # we're going to higher D's in this plot, so we increase Ψ'(0) to avoid underflows.

V = np.ones((K,D))/np.sqrt(D) # unit length vectors, some functions need vector-valued inputs
μs_norm = np.linspace(0, 1, K+2)[1:-1]
Ψμ, gradΨμ = Ψ(μs_norm.reshape(-1,1)*V, D, Ψ0=Ψ0, t0=t0, return_grad=True, solve_delta=True)
dΨμ = np.linalg.norm(gradΨμ,axis=-1)

Ψμ_app, gradΨμ_app = Ψ(μs_norm.reshape(-1,1)*V, D, Ψ0=[0., 0], t0=t0, return_grad=True, solve_delta=True)
dΨμ_app = np.linalg.norm(gradΨμ_app,axis=-1)


def rec(μ, df, D):
    (D-1.) * μ / (1. - μ**2 - 1./ df)
def f0(μ):
    return μ

f = f0
df = grad(f)

In [ ]:
from jax import grad

def rec(μ, df, D):
    return (D-1.) * μ / (1. - μ**2 - 1./ df(μ))
def rec_sum(μ, df, D):
    return (D-1.) * (μ / (1. - μ**2 - 1./ df(μ))).sum()

def f0(μ):
    return (μ*(D-μ**2)/(1-μ**2))

def f0_sum(μ):
    return (μ*(D-μ**2)/(1-μ**2)).sum()

df0 = grad(f0_sum)

f1 = rec(μs_norm, df0, D)
f1_rec = rec_sum(μs_norm, df0, D)

def df1(μ):
    return (D-1) * (1. + μ**2 + 1/df0(μ) + grad(f0_sum)(μ)/df0(μ)**2) / (1-μ**2 + 1/df0(μ))**2
f2 = rec(μs_norm, df1, D)

def df1_sum(μ):
    return (D-1) * ((1. + μ**2 + 1/df0(μ) + grad(f0_sum)(μ)/df0(μ)**2) / (1-μ**2 + 1/df0(μ))**2).sum()
def df2(μ):
    return (D-1) * (1. + μ**2 + 1/df1(μ) + grad(df1_sum)(μ)/df1(μ)**2) / (1-μ**2 + 1/df1(μ))**2
f3 = rec(μs_norm, df2, D)

plt.subplot(1,2,1)
plt.semilogy(μs_norm, dΨμ, label='num')
plt.semilogy(μs_norm, f0(μs_norm), label='test0')
plt.semilogy(μs_norm, f1, label='test1')
plt.semilogy(μs_norm, f2, label='test2')
plt.semilogy(μs_norm, f3, label='test3')
plt.legend()

plt.subplot(1,2,2)
plt.semilogy(μs_norm, np.abs(dΨμ-dΨμ_app), label='current')
plt.semilogy(μs_norm, np.abs(dΨμ-f0(μs_norm)), label='test0')
plt.semilogy(μs_norm, np.abs(dΨμ-f1), label='test1')
plt.semilogy(μs_norm, np.abs(dΨμ-f2), label='test2')
plt.semilogy(μs_norm, np.abs(dΨμ-f3), label='test3')
plt.legend()


# On the initial condition
 - $\psi'(0) = 0$ means that $\psi''(0) = \frac{0}{0}$ is ill-defined.
 - in practice we start from 'something small', but not small enough to underflow with given numerical precision (64-bit)
 - typically, $\psi'(0) = 1e-6$ for $D < 100$, $\psi'(0) = 1e-4$ for $D\geq 1000$. With much larger $D$, might have to go even smaller
 - here we look at $D=2$ and $D=100$ to show that solutions quickly approach each other, i.e. error from $\psi'(0) > 0$ is mostly limited to small $||\mu||$.
 
 

In [ ]:
from vMFne.logpartition import gradΦ, invgradΦ, vMF_entropy_Φ, logχ, log_besseli, banerjee_44
from vMFne.negentropy import Ψ, Ψ_base, gradΨ, dΨ_base
from scipy.special import loggamma
import mpmath

import numpy as np
import matplotlib.pyplot as plt

dΨ0 = [1e-6, 1e-2] # values to try for Ψ'(0).

t0 = 0.0 # start numerical integration from ||μ||=t0.
K = 100  # number of evaluation points between 0 and max{||μ||} for this experiment.

D = 2
V = np.ones((K,D))/np.sqrt(D) # unit length vectors, some functions need vector-valued inputs

Ψμs = np.zeros((len(dΨ0), K))
dΨμs = np.zeros((len(dΨ0), K))
μs_norm = np.linspace(0, 0.1, K+2)[1:-1]
for i in range(len(dΨ0)):
    Ψ0 = [0., dΨ0[i]]
    Ψμ, dΨμ = Ψ(μs_norm.reshape(-1,1)*V, D, Ψ0=Ψ0, t0=t0, return_grad=True)
    Ψμs[i] = Ψμ
    dΨμs[i] = np.linalg.norm(dΨμ,axis=-1)

fontsize=14
ticksize=12

plt.figure(figsize=(8,7))
    
plt.subplot(2,2,1)
plt.semilogy(μs_norm, dΨμs.T)
plt.ylabel(r"$|| \nabla\Psi(\mu) ||$",fontsize=fontsize)
plt.title('D = 2',fontsize=fontsize)
plt.yticks(fontsize=ticksize)
plt.xticks(fontsize=ticksize)

plt.subplot(2,2,3)
plt.semilogy(μs_norm, np.diff(Ψμs,axis=0).T, 'k:')
plt.semilogy(μs_norm, np.diff(dΨμs,axis=0).T, 'k') 
plt.xlabel(r"$|| \mu ||$",fontsize=fontsize)
plt.ylabel("abs. difference",fontsize=fontsize)
plt.yticks(fontsize=ticksize)
plt.xticks(fontsize=ticksize)


D = 100
V = np.ones((K,D))/np.sqrt(D)

Ψμs = np.zeros((len(dΨ0), K))
dΨμs = np.zeros((len(dΨ0), K))
μs_norm = np.linspace(0, 0.001, K+2)[1:-1]
for i in range(len(dΨ0)):
    Ψ0 = [0., dΨ0[i]]
    Ψμ, dΨμ = Ψ(μs_norm.reshape(-1,1)*V, D, Ψ0=Ψ0, t0=t0, return_grad=True)
    Ψμs[i] = Ψμ
    dΨμs[i] = np.linalg.norm(dΨμ,axis=-1)

plt.subplot(2,2,2)
plt.semilogy(μs_norm, dΨμs.T)
plt.legend([r"$||\nabla\Psi(0)||=10^{-6}$", r"$||\nabla\Psi(0)||=10^{-2}$"], frameon=False,fontsize=fontsize)
plt.title('D = 100',fontsize=fontsize)
plt.yticks(fontsize=ticksize)
plt.xticks([0, 0.0005, 0.001], fontsize=ticksize)

plt.subplot(2,2,4)
plt.semilogy(μs_norm, np.diff(dΨμs,axis=0).T, 'k') 
plt.semilogy(μs_norm, np.diff(Ψμs,axis=0).T, 'k:')
plt.legend([r"$||\nabla\Psi(\mu)||$", r"$\Psi(\mu)$"], frameon=False,fontsize=fontsize)
plt.xlabel(r"$|| \mu ||$",fontsize=fontsize)
plt.yticks([1e-4, 1e-8, 1e-12], fontsize=ticksize)
plt.xticks([0, 0.0005, 0.001], fontsize=ticksize)

for i in range(4):
    ax = plt.subplot(2,2,i+1)
    ax.spines[['right', 'top']].set_visible(False)

plt.savefig('figS1.pdf', bbox_inches='tight')
plt.show()


# Data application - Bregman clustering

- we want to showacse the use of the negative entropy and its gradient for the mean-parameter representation of the von-Mises Fisher distribiution
- to this end, we reproduce previous work (Banerjee et al. 2005) on document clustering in high-dimensional spherical feature spaces.
- this line of work uses mixtures of von-Mises Fisher ('movMF') distributions to infer latent cluster assignments for individual documents mapped onto a spherical feature space. 
- we compare the classical movFM algorithm (in natural-parameter space) against Bregman Clustering (in mean-parameter space), and do this over two different variants of movFM models (tied and independent variances across mixture components). 
- datasets are the classic datasets ['20-newsgroups'](https://scikit-learn.org/0.19/datasets/twenty_newsgroups.html) and ['classic3'](http://ir.dcs.gla.ac.uk/resources/test_collections/), as well as subsampled variants thereof. 


## run all algorithms for spherical clustering on both datasets

In [ ]:
from exps import run_all_classic3

version = '1'
classic300 = True

run_all_classic3(fn_root='results/classic300_', n_repets=10, K_range=[2,3,4,5,6,7,8,9,10,11], 
                 seed=0, max_iter=100, κ_max=10000., Ψ0=[None, 0.], version=version, 
                 classic300=classic300, verbose=True, min_df=2, max_df=0.15)


In [ ]:
from exps import run_all_news20

version = '1'
news20_small = True

run_all_news20(fn_root='results/news20small_', n_repets=10, K_range=[4,8,12,16,20,24,28,32,36,40], 
                 seed=0, max_iter=100, κ_max=10000., Ψ0=[None, 0.], version=version, 
                 news20_small=news20_small, verbose=True, min_df=2, max_df=0.15)


In [ ]:
from exps import run_all_classic3

version = '1'
classic300 = False

run_all_classic3(fn_root='results/classic3_', n_repets=10, K_range=[2,3,4,5,6,7,8,9,10,11], 
                 seed=0, max_iter=100, κ_max=10000., Ψ0=[None, 0.], version=version, 
                 classic300=classic300, verbose=True)


In [ ]:
from exps import run_all_news20

version = '1'
news20_small = False

run_all_news20(fn_root='results/news20_', n_repets=10, K_range=[4,8,12,16,20,24,28,32,36,40], 
                 seed=0, max_iter=100, κ_max=10000., Ψ0=[None, 0.], version=version, 
                 news20_small=news20_small, verbose=True)


# Bregman clustering contd'
## compute & store (normalized) mutual information across all algorithms and datasets
- needs previous cells to run first.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import metrics

from exps import load_classic3_sklearn, load_news20_sklearn
from vMFne.bregman_clustering import posterior_marginal_vMF_mixture_Ψ
from vMFne.moVMF import posterior_marginal_vMF_mixture_Φ

Ψ0 = [None, 0.]

# Calculate MI between two clusterings
def mi(class_true, class_est):
    cont = metrics.cluster.contingency_matrix(class_true, class_est, sparse=True).astype(np.float64, copy=False)
    mi = metrics.cluster.mutual_info_score(class_true, class_est, contingency=cont)
    return mi

def get_MIs(fn_rootroot, fn_roots, n_repets, K_range, class_true, NMI=False):
    MIs = np.zeros((len(fn_roots), n_repets, len(K_range)))
    comp_mi = metrics.normalized_mutual_info_score if NMI else mi
    for f,fn_root in enumerate(fn_roots):
        for k,K in enumerate(K_range):
            if f == 0:
                fn = fn_rootroot + fn_root + str(K) + '.npy'
                out = np.load(fn, allow_pickle=True)
                for i in range(n_repets):
                    class_est = out[i]
                    MIs[f,i,k] =  comp_mi(class_true.flatten(), class_est.flatten())
            elif f in [1,2,5,6]:
                fn = fn_rootroot + fn_root + str(K) + '.npz'
                out = np.load(fn, allow_pickle=True)['out'].tolist()
                for i in range(n_repets):
                    ph_x_μ, _ = posterior_marginal_vMF_mixture_Ψ(X,out['w'][i],out['μs'][i], Ψ0=Ψ0)
                    class_est = np.argmax(ph_x_μ,axis=1)
                    MIs[f,i,k] = comp_mi(class_true.flatten(), class_est.flatten()) 
            elif f in [3,4,7,8]:
                fn = fn_rootroot + fn_root + str(K) + '.npz'
                out = np.load(fn, allow_pickle=True)['out'].tolist()
                for i in range(n_repets):
                    ph_x_η, _ = posterior_marginal_vMF_mixture_Φ(X,out['w'][i],out['ηs'][i])
                    class_est = np.argmax(ph_x_η,axis=1)
                    MIs[f,i,k] = comp_mi(class_true.flatten(), class_est.flatten()) 
    return MIs

def get_fn_roots(n_repets, version):
    return [f'spkmeans_{n_repets}repets_seed_0_v{version}__K_',
            f'softBregClust_{n_repets}repets_seed_0_no_tying__v{version}__K_',
            f'softBregClust_{n_repets}repets_seed_0_with_tying__v{version}__K_',
            f'softmovMF_{n_repets}repets_seed_0_no_tying__v{version}__K_',
            f'softmovMF_{n_repets}repets_seed_0_with_tying__v{version}__K_',
            f'hardBregClust_{n_repets}repets_seed_0_no_tying__v{version}__K_',
            f'hardBregClust_{n_repets}repets_seed_0_with_tying__v{version}__K_',
            f'hardmovMF_{n_repets}repets_seed_0_no_tying__v{version}__K_',
            f'hardmovMF_{n_repets}repets_seed_0_with_tying__v{version}__K_']


In [ ]:
fn_rootroot = 'results/classic3_'
K_range = np.arange(2,12,1)
n_repets = 10
version = 1
seed = 0

X, labels, dictionary = load_classic3_sklearn()
class_true = sum([ (1.*i) * (labels==np.unique(labels)[i]) for i in range(len(np.unique(labels)))])

fn_roots = get_fn_roots(n_repets, version)
MIs_classic3 = get_MIs(fn_rootroot, fn_roots, n_repets, K_range, class_true, NMI=True)

plt.figure(figsize=(6,4))
clrs = ['b', 'magenta', 'magenta', 'black', 'black', 'orange', 'orange', 'green', 'green' ]
lnstl = ['-', '--', '-', '--', '-', '--', '-', '--', '-']
mrkrs = ['o','o','o','.','.','o','o','.','.']
for f in range(len(fn_roots)):
    plt.plot(K_range, MIs_classic3.mean(axis=1)[f], label=fn_roots[f], 
             color=clrs[f], linestyle=lnstl[f], marker=mrkrs[f])
plt.xlabel('K')
plt.ylabel('NMI')
plt.show()

np.save(f'{fn_rootroot}NMIs_{n_repets}repets_seed_0_v{version}__K{min(K_range)}_{max(K_range)}', MIs_classic3)
np.save(f'{fn_rootroot}NMIs_{n_repets}repets_seed_0_v{version}__K{min(K_range)}_{max(K_range)}', 
        {'MIs' : MIs_classic3,
         'algorithms' : fn_roots,
         'K_range' : K_range,
         'seed' : seed,
         'n_repets' : n_repets,
        })


In [ ]:
fn_rootroot = 'results/classic300_'
K_range = np.arange(2,12,1)
n_repets = 10
version = 1
seed = 0

X, labels, dictionary = load_classic3_sklearn(classic300=True, min_df=2, max_df=0.15)
class_true = sum([ (1.*i) * (labels==np.unique(labels)[i]) for i in range(len(np.unique(labels)))])

fn_roots = get_fn_roots(n_repets, version)
MIs_classic300 = get_MIs(fn_rootroot, fn_roots, n_repets, K_range, class_true, NMI=True)

plt.figure(figsize=(6,4))
clrs = ['b', 'magenta', 'magenta', 'black', 'black', 'orange', 'orange', 'green', 'green' ]
lnstl = ['-', '--', '-', '--', '-', '--', '-', '--', '-']
mrkrs = ['o','o','o','.','.','o','o','.','.']
for f in range(len(fn_roots)):
    plt.plot(K_range, MIs_classic300.mean(axis=1)[f], label=fn_roots[f], 
             color=clrs[f], linestyle=lnstl[f], marker=mrkrs[f])
plt.xlabel('K')
plt.ylabel('NMI')
#plt.legend()
plt.show()

np.save(f'{fn_rootroot}NMIs_{n_repets}repets_seed_0_v{version}__K{min(K_range)}_{max(K_range)}', MIs_classic300)
np.save(f'{fn_rootroot}NMIs_{n_repets}repets_seed_0_v{version}__K{min(K_range)}_{max(K_range)}', 
        {'MIs' : MIs_classic300,
         'algorithms' : fn_roots,
         'K_range' : K_range,
         'seed' : seed,
         'n_repets' : n_repets,
        })


In [ ]:
fn_rootroot = 'results/news20_'
K_range = np.arange(4,41,4)
n_repets = 10
version = 1
seed = 0

X, labels, dictionary = load_news20_sklearn()
class_true = sum([ (1.*i) * (labels==np.unique(labels)[i]) for i in range(len(np.unique(labels)))])

fn_roots = get_fn_roots(n_repets, version)
MIs_news20 = get_MIs(fn_rootroot, fn_roots, n_repets, K_range, class_true, NMI=True)

plt.figure(figsize=(6,4))
clrs = ['b', 'magenta', 'magenta', 'black', 'black', 'orange', 'orange', 'green', 'green' ]
lnstl = ['-', '--', '-', '--', '-', '--', '-', '--', '-']
mrkrs = ['o','o','o','.','.','o','o','.','.']
for f in range(len(fn_roots)):
    plt.plot(K_range, MIs_news20.mean(axis=1)[f], label=fn_roots[f], 
             color=clrs[f], linestyle=lnstl[f], marker=mrkrs[f])
plt.xlabel('K')
plt.ylabel('NMI')
#plt.legend()
plt.show()

np.save(f'{fn_rootroot}NMIs_{n_repets}repets_seed_0_v{version}__K{min(K_range)}_{max(K_range)}', MIs_news20)
np.save(f'{fn_rootroot}NMIs_{n_repets}repets_seed_0_v{version}__K{min(K_range)}_{max(K_range)}', 
        {'MIs' : MIs_news20,
         'algorithms' : fn_roots,
         'K_range' : K_range,
         'seed' : seed,
         'n_repets' : n_repets,
        })


In [ ]:
fn_rootroot = 'results/news20small_'
K_range = np.arange(4,41,4)
n_repets = 10
version = 1
seed = 0
X, labels, dictionary = load_news20_sklearn(news20_small=True, min_df=2, max_df=0.15)
class_true = sum([ (1.*i) * (labels==np.unique(labels)[i]) for i in range(len(np.unique(labels)))])

fn_roots = get_fn_roots(n_repets, version)
MIs_news20small = get_MIs(fn_rootroot, fn_roots, n_repets, K_range, class_true, NMI=True)

plt.figure(figsize=(6,4))
clrs = ['b', 'magenta', 'magenta', 'black', 'black', 'orange', 'orange', 'green', 'green' ]
lnstl = ['-', '--', '-', '--', '-', '--', '-', '--', '-']
mrkrs = ['o','o','o','.','.','o','o','.','.']
for f in range(len(fn_roots)):
    plt.plot(K_range, MIs_news20small.mean(axis=1)[f], label=fn_roots[f], 
             color=clrs[f], linestyle=lnstl[f], marker=mrkrs[f])
plt.xlabel('K')
plt.ylabel('NMI')
#plt.legend()
plt.show()

np.save(f'{fn_rootroot}NMIs_{n_repets}repets_seed_0_v{version}__K{min(K_range)}_{max(K_range)}', MIs_news20small)
np.save(f'{fn_rootroot}NMIs_{n_repets}repets_seed_0_v{version}__K{min(K_range)}_{max(K_range)}', 
        {'MIs' : MIs_news20small,
         'algorithms' : fn_roots,
         'K_range' : K_range,
         'seed' : seed,
         'n_repets' : n_repets,
        })


# Bregman clustering contd'
## make results figure

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

alg_names = ['spkmeans',
             'softBregClust',
             'softBregClust',
             'softmovMF',
             'softmovMF',
             'hardBregClust',
             'hardBregClust',
             'hardmovMF',
             'hardmovMF']

alg_names_disp = ['spkmeans',
             'soft Breg.Clust.',
             'soft Breg.Clust.',
             'soft nat-par. EM',
             'soft nat-par. EM',
             'hard Breg.Clust.',
             'hard Breg.Clust.',
             'hard nat-par. EM',
             'hard nat-par. EM']

dataset_names_dips = ['classic3', 'classic300', 'news20', 'news20-small']

Kminmax= [[2,11], [4,40]]
true_Ks = [3, 20]

n_repets = 10
version = 1
seed = 0

clrs = ['b', 'magenta', 'magenta', 'black', 'black', 'orange', 'orange', 'green', 'green' ]
lnstl = ['-', '--', '-', '--', '-', '--', '-', '--', '-']
mrkrs = ['.','o','o','.','.','o','o','.','.']

fontsize=14
ticksize=12

fn_rootroots = [['results/classic3_', 'results/classic300_'],
                ['results/news20_', 'results/news20small_']]

plt.figure(figsize=(8,7))
for j in range(len(fn_rootroots)): # over datasets 

    for i, fn_rootroot in enumerate(fn_rootroots[j]): # over algorithms

        plt.subplot(2,2,2*i + j + 1)
        out = dict(np.load(
            f'{fn_rootroot}NMIs_{n_repets}repets_seed_0_v{version}__K{Kminmax[j][0]}_{Kminmax[j][1]}.npy',
            allow_pickle=True).tolist())

        for f in range(len(alg_names)):
            if alg_names[f] == 'spkmeans':
                pass
            elif lnstl[f] == '-':
                plt.plot(out['K_range'], out['MIs'].mean(axis=1)[f], label=alg_names_disp[f], 
                         color=clrs[f], linestyle=lnstl[f], marker=mrkrs[f])
            else:
                plt.plot(out['K_range'], out['MIs'].mean(axis=1)[f], 
                         color=clrs[f], linestyle=lnstl[f], marker=mrkrs[f])
        plt.title(dataset_names_dips[2*j+i], fontsize=fontsize, y=0.97)
        plt.xlim([Kminmax[j][0], Kminmax[j][1]])
        plt.plot([true_Ks[j], true_Ks[j]], [0.9*out['MIs'].mean(axis=1).min(), 1.02*out['MIs'].mean(axis=1).max()],
                 color='gray', alpha=0.5)
        plt.ylim([0.9*out['MIs'].mean(axis=1).min(), 1.05*out['MIs'].mean(axis=1).max()])
        plt.yticks(fontsize=ticksize)
        plt.xticks(fontsize=ticksize)
        if i == 1 and j == 0:
            plt.legend(fontsize=ticksize, frameon=False)
        if j == 0:
            plt.ylabel('Norm. mutual information', fontsize=fontsize)
        if i == 1:
            plt.xlabel('# of clusters K', fontsize=fontsize)

for i in range(4):
    ax = plt.subplot(2,2,i+1)
    ax.spines[['right', 'top']].set_visible(False)

plt.savefig('fig3.pdf', bbox_inches='tight')
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from exps import load_news20_sklearn
X, labels, dictionary = load_news20_sklearn(news20_small=False, min_df=6, max_df=0.15)

In [ ]:
#run_softBregmanClustering(fn, X, K_range, n_repets, max_iter=100, seed=0, verbose=False, 
#                  tie_norms=False, μ_norm_max=0.99, init_with_spkmeans=False, Ψ0=[None, 0.])

from vMFne.bregman_clustering import softBregmanClustering_vMF, hardBregmanClustering_vMF

K_range = [20]
n_repets = 3
max_iter = 100
seed = 0
tie_norms=False
Ψ0=[None, 0.]
μ_norm_max=0.99
verbose=True
init_with_spkmeans=False

assert np.allclose(np.linalg.norm(X,axis=-1), 1.)
N,D = X.shape

for K in K_range: # for each number of clusters ...
    if verbose:
        print('K='+str(K))
    all_μs, all_w, all_LL, all_c  = [], [], [], []
    for ii in range(n_repets): # for n_repets different random initializations 
        np.random.seed(seed+ii)
        if init_with_spkmeans:
            _, w, c = spherical_kmeans(X, K=K, max_iter=max_iter, verbose=False)
            all_c.append(1 * c)            
            μs = np.stack([X[c==k].mean(axis=0) for k in range(K)],axis=0)
        else:
            μs, w = None, None
        μs, w, LL = softBregmanClustering_vMF(X, K=K, max_iter=max_iter, verbose=False, Ψ0=Ψ0,
                          μs_init=μs, w_init=w, tie_norms=tie_norms, μ_norm_max=μ_norm_max)
        all_μs.append(1. * μs)
        all_w.append(1. * w)
        all_LL.append(1. * LL)

        if verbose:
            print(' - ' + str(ii+1) + '/' + str(n_repets))
LL_Ψ = 1. * np.stack(all_LL)
w_Ψ = 1. * np.stack(all_w)

In [ ]:
from vMFne.moVMF import softMoVMF, hardMoVMF, posterior_marginal_vMF_mixture_Φ

#def run_softmovMF(fn, X, K_range, n_repets, max_iter=100, seed=0, verbose=False, 
#                  tie_norms=False, κ_max=np.inf, init_with_spkmeans=False):

κ_max=np.inf

assert np.allclose(np.linalg.norm(X,axis=-1), 1.)
N,D = X.shape

for K in K_range: # for each number of clusters ...
    if verbose:
        print('K='+str(K))
    all_ηs, all_w, all_LL, all_c  = [], [], [], []
    for ii in range(n_repets): # for n_repets different random initializations 
        np.random.seed(seed+ii)
        if init_with_spkmeans:
            _, w, c = spherical_kmeans(X, K=K, max_iter=max_iter, verbose=False)
            all_c.append(1 * c)            
            μs = np.stack([X[c==k].mean(axis=0) for k in range(K)],axis=0)
            ηs = gradΨ(μs, D=D, Ψ0=1e-6)
        else:
            ηs, w = None, None
        ηs, w, LL = softMoVMF(X, K=K, max_iter=max_iter, verbose=False, 
                              ηs_init=ηs, w_init=w, tie_norms=tie_norms, κ_max=κ_max)
        all_ηs.append(1. * ηs)
        all_w.append(1. * w)
        all_LL.append(1. * LL)

        if verbose:
            print(' - ' + str(ii+1) + '/' + str(n_repets))
LL_Φ = 1. * np.stack(all_LL)

In [ ]:
ii = 3

plt.subplot(1,2,1)
plt.plot((LL_Ψ[:ii]-LL_Ψ[:ii,:1]).T, linewidth=2.5)
plt.plot((LL_Φ[:ii]-LL_Φ[:ii,:1]).T, 'k--')

plt.subplot(1,2,2)
plt.plot(((LL_Ψ[:ii]-LL_Ψ[:ii,:1]).T - (LL_Φ[:ii]-LL_Φ[:ii,:1]).T))
plt.show()

for i in range(len(all_LL)):
    plt.plot(((LL_Ψ[i]-LL_Ψ[i,:1]).T - (LL_Φ[i]-LL_Φ[i,:1]).T), label=str(i))
plt.legend()
plt.show()


In [ ]:
from vMFne.moVMF import posterior_marginal_vMF_mixture_Φ
from vMFne.bregman_clustering import posterior_marginal_vMF_mixture_Ψ

for i in range(ii):
    post_Φ, _ = posterior_marginal_vMF_mixture_Φ(X=X,w=all_w[i],ηs=all_ηs[i])
    post_Ψ, _ = posterior_marginal_vMF_mixture_Ψ(X=X,w=w_Ψ[i],μs=all_μs[i], Ψ0=[0.,0.])
    plt.loglog(post_Φ.flatten(), post_Ψ.flatten(), '.')
    plt.show()


In [ ]:
from vMFne.moVMF import posterior_marginal_vMF_mixture_Φ
from vMFne.bregman_clustering import posterior_marginal_vMF_mixture_Ψ

for i in range(ii):
    post_Φ, _ = posterior_marginal_vMF_mixture_Φ(X=X,w=all_w[i],ηs=all_ηs[i])
    post_Ψ, _ = posterior_marginal_vMF_mixture_Ψ(X=X,w=w_Ψ[i],μs=all_μs[i], Ψ0=[0.,0.])
    plt.plot(np.argmax(post_Φ,axis=1).flatten(), np.argmax(post_Ψ,axis=1).flatten(), '.')
    plt.show()


In [ ]:
import numpy as np
from exps import load_classic3_sklearn 

def get_fn_roots(n_repets, version):
    return [f'spkmeans_{n_repets}repets_seed_0_v{version}__K_',
            f'softBregClust_{n_repets}repets_seed_0_no_tying__v{version}__K_',
            f'softBregClust_{n_repets}repets_seed_0_with_tying__v{version}__K_',
            f'softmovMF_{n_repets}repets_seed_0_no_tying__v{version}__K_',
            f'softmovMF_{n_repets}repets_seed_0_with_tying__v{version}__K_',
            f'hardBregClust_{n_repets}repets_seed_0_no_tying__v{version}__K_',
            f'hardBregClust_{n_repets}repets_seed_0_with_tying__v{version}__K_',
            f'hardmovMF_{n_repets}repets_seed_0_no_tying__v{version}__K_',
            f'hardmovMF_{n_repets}repets_seed_0_with_tying__v{version}__K_']


In [ ]:
all_norms = np.zeros((4,8,11))

In [ ]:
fn_rootroot = 'results/news20_'
K_range = np.arange(4,41,4)
n_repets = 10
version = 0
seed = 0

fn_roots = get_fn_roots(n_repets, version)
for f,fn_root in enumerate(fn_roots):
    for k,K in enumerate(K_range):
        if f == 0:
            pass
        elif f in [1,2,5,6]:
            fn = fn_rootroot + fn_root + str(K) + '.npz'
            out = np.load(fn, allow_pickle=True)['out'].tolist()
            print(np.linalg.norm(out['μs'],axis=-1).mean())
            all_norms[0,f-1,k] = np.linalg.norm(out['μs'],axis=-1).mean()
        elif f in [3,4,7,8]:
            fn = fn_rootroot + fn_root + str(K) + '.npz'
            out = np.load(fn, allow_pickle=True)['out'].tolist()
            #print(fn)
            #print(np.linalg.norm(out['ηs'],axis=-1).mean())


In [ ]:
fn_rootroot = 'results/news20small_'
K_range = np.arange(4,41,4)
n_repets = 10
version = 0
seed = 0

fn_roots = get_fn_roots(n_repets, version)
for f,fn_root in enumerate(fn_roots):
    for k,K in enumerate(K_range):
        if f == 0:
            pass
        elif f in [1,2,5,6]:
            fn = fn_rootroot + fn_root + str(K) + '.npz'
            out = np.load(fn, allow_pickle=True)['out'].tolist()
            print(np.linalg.norm(out['μs'],axis=-1).mean())
            all_norms[1,f-1,k] = np.linalg.norm(out['μs'],axis=-1).mean()
        elif f in [3,4,7,8]:
            fn = fn_rootroot + fn_root + str(K) + '.npz'
            out = np.load(fn, allow_pickle=True)['out'].tolist()
            #print(fn)
            #print(np.linalg.norm(out['ηs'],axis=-1).mean())


In [ ]:
fn_rootroot = 'results/classic3_'
K_range = np.arange(2,12,1)
n_repets = 10
version = 0
seed = 0

fn_roots = get_fn_roots(n_repets, version)
for f,fn_root in enumerate(fn_roots):
    for k,K in enumerate(K_range):
        if f == 0:
            pass
        elif f in [1,2,5,6]:
            fn = fn_rootroot + fn_root + str(K) + '.npz'
            out = np.load(fn, allow_pickle=True)['out'].tolist()
            print(np.linalg.norm(out['μs'],axis=-1).mean())
            all_norms[2,f-1,k] = np.linalg.norm(out['μs'],axis=-1).mean()
        elif f in [3,4,7,8]:
            fn = fn_rootroot + fn_root + str(K) + '.npz'
            out = np.load(fn, allow_pickle=True)['out'].tolist()
            #print(fn)
            #print(np.linalg.norm(out['ηs'],axis=-1).mean())


In [ ]:
fn_rootroot = 'results/classic300_'
K_range = np.arange(2,12,1)
n_repets = 10
version = 0
seed = 0

fn_roots = get_fn_roots(n_repets, version)
for f,fn_root in enumerate(fn_roots):
    for k,K in enumerate(K_range):
        if f == 0:
            pass
        elif f in [1,2,5,6]:
            fn = fn_rootroot + fn_root + str(K) + '.npz'
            out = np.load(fn, allow_pickle=True)['out'].tolist()
            print(np.linalg.norm(out['μs'],axis=-1).mean())
            all_norms[3,f-1,k] = np.linalg.norm(out['μs'],axis=-1).mean()
        elif f in [3,4,7,8]:
            fn = fn_rootroot + fn_root + str(K) + '.npz'
            out = np.load(fn, allow_pickle=True)['out'].tolist()
            #print(fn)
            #print(np.linalg.norm(out['ηs'],axis=-1).mean())


In [ ]:
idx = np.array([0,1,4,5])
plt.plot(all_norms[0,idx[0]].T, 'kx-') # news20, softBreg free
plt.plot(all_norms[0,idx[1]].T, 'k.-') # news20, softBreg tied
plt.plot(all_norms[0,idx[2]].T, 'ko-') # news20, hardBreg free
plt.plot(all_norms[0,idx[3]].T, 'k-') # news20, hardBreg tied
plt.plot(all_norms[1,idx].T, 'gray')
plt.plot(all_norms[2,idx[0]].T, 'rx-')
plt.plot(all_norms[2,idx[1]].T, 'r.-')
plt.plot(all_norms[2,idx[2]].T, 'ro-')
plt.plot(all_norms[2,idx[3]].T, 'r*-')
plt.plot(all_norms[3,idx].T, 'orange')
plt.show()